In [1]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.9 MB/s eta 0:00:0000:0100:01


In [2]:
!pip -q install transformers huggingface_hub sentence_transformers

In [3]:
!pip install faiss

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [4]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 14.8 MB/s eta 0:00:0000:0100:01


In [5]:
!pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 78.7 MB/s eta 0:00:0000:010:01


In [6]:
import torch
from transformers import BitsAndBytesConfig

# Check if CUDA (GPU) is available and set the device accordingly
device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'

In [7]:
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import WebBaseLoader

# Define a list of URLs from which you want to load documents
urls = [
    "https://www.deeplearningbook.org/contents/part_basics.html",
    "https://www.deeplearningbook.org/contents/linear_algebra.html",
    "https://www.deeplearningbook.org/contents/prob.html",
    "https://www.deeplearningbook.org/contents/numerical.html",
    "https://www.deeplearningbook.org/contents/ml.html"
]

# Create an empty list to store the loaded documents
documents = []

# Load documents from each URL
for url in urls:
    loader = WebBaseLoader(url)
    documents.extend(loader.load())

# Split text from PDF into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                               chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# Load embeddings model
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                   model_kwargs={'device': 'cuda'})

# Build and persist FAISS vector store
vectorstore = FAISS.from_documents(texts, embeddings)
vectorstore.save_local('vectorstore/db_faiss')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Batches:   0%|          | 0/21 [00:00<?, ?it/s]

In [8]:
qa_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
Question: {question}
Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [9]:
from langchain.llms import CTransformers

# Local CTransformers wrapper for Llama-2-7B-Chat
llm = CTransformers(model='TheBloke/Llama-2-7B-Chat-GGML',
                    model_type='llama', # Model type Llama
                    config={'max_new_tokens': 256,
                            'temperature': 0.01})

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Wrap prompt template in a PromptTemplate object
def set_qa_prompt():
    prompt = PromptTemplate(template=qa_template,
                            input_variables=['context', 'question'])
    return prompt


# Build RetrievalQA object
def build_retrieval_qa(llm, prompt, vectordb):
    dbqa = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=vectordb.as_retriever(search_kwargs={'k':2}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt})
    return dbqa

In [11]:
import timeit

def setup_dbqa():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cuda'})
    vectordb = FAISS.load_local('vectorstore/db_faiss', embeddings)
    qa_prompt = set_qa_prompt()
    dbqa = build_retrieval_qa(llm, qa_prompt, vectordb)

    return dbqa

def main(input_question):
    start = timeit.default_timer()  # Start timer
    
    print(device)
    
    # Setup QA object
    dbqa = setup_dbqa()

    try:
        # Parse input from the function argument into QA object
        response = dbqa({'query': input_question})
        end = timeit.default_timer()  # End timer

        # Print document QA response
        if "result" in response:
            print(f'\nAnswer: {response["result"]}')
        else:
            print("No answer found.")
        print('=' * 50)  # Formatting separator

        # Process source documents for better display
        source_docs = response.get('source_documents', [])
        for i, doc in enumerate(source_docs):
            print(f'\nSource Document {i + 1}\n')
            print(f'Source Text: {doc.page_content}')
            print(f'Document Name: {doc.metadata["source"]}')
            print('=' * 50)  # Formatting separator

        # Display time taken for CPU inference
        total_time = end - start
        print(f"Time to retrieve response: {total_time} seconds")

    except Exception as e:
        print("Error occurred while processing the question:", e)

# Replace the input_question variable with your desired question
input_question = "Linear algebra is"
main(input_question)

cuda:0


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Answer: Linear algebra is a branch of mathematics that deals with linear equations and their transformations. It includes the study of vectors, vector operations, and matrix multiplication.

Source Document 1

Source Text: Chapter 2Linear AlgebraLinear algebra is a branch of mathematics that is widely used throughout scienceand engineering. Yet because linear algebra is a form of continuous rather thandiscrete mathematics, many computer scientists have little experience with it. Agood understanding of linear algebra is essential for understanding and workingwith many machine learning algorithms, especially deep learning algorithms. Wetherefore precede our introduction to deep learning with a focused presentation
Document Name: https://www.deeplearningbook.org/contents/linear_algebra.html

Source Document 2

Source Text: CHAPTER 2. LINEAR ALGEBRAdeﬁne a vector by writing out its elements in the text inline as a row matrix, thenusing the transpose operator to turn it into a standard col